# Making chloropleth maps in Altair

Here's a quick example of how to make a chloropleth map in Altair.  In this example, we'll work with a fairly large data set of baby names in France from 1900-2019, broken down by department.

To work with geographical data, we'll use the `geopandas`, which loads `pandas` dataframes, but with support for geographical outlines in the `geojson` format.  You can use these dataframes just as you would a regular `pandas` dataframe, but they will include that extra geographical outline data.

To get started, we'll need to import our libraries.

In [2]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
alt.data_transformers.enable('json') # Let Altair/Vega-Lite work with large data sets

pass

# Reading our names data

Now, let's read in our dataset.  The exported data is in CSV format, but with a `;` separator instead of commas.  The INSEE data collapses rare names or where department-level information has been elided (presumably to protect individuals with uncommon names or who were one of the only ones born with that name in a given year).  We'll strip those out.

In [3]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
1538637,1,STÉPHANE,1962,74,28
832140,1,JEAN-PHILIPPE,1971,17,19
62450,1,ALBERT,1922,76,87
1853912,2,ANAËLLE,2015,47,5
950589,1,KEVIN,1994,87,55


# Loading map data

Next, let's load some map data of regions in France using `geopandas`.  These map data come from the [INSEE] and [IGN] and were processed into the `geojson` format we'll need to work with by [Grégoire David].  Here's the [github] repository.

In this example, we'll work with the simplified departments tiles for the Hexagon, but that repository contains higher-resolution versions, the DOM-TOM, and more.

[Grégoire David]: https://gregoiredavid.fr
[INSEE]: http://www.insee.fr/fr/methodes/nomenclatures/cog/telechargement.asp
[IGN]: https://geoservices.ign.fr/adminexpress
[github]: https://github.com/gregoiredavid/france-geojson/

In [4]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
12,13,Bouches-du-Rhône,"POLYGON ((4.73906 43.92406, 4.82174 43.91283, ..."
56,56,Morbihan,"MULTIPOLYGON (((-3.42179 47.62000, -3.44067 47..."
77,77,Seine-et-Marne,"POLYGON ((2.57166 48.69201, 2.56880 48.70722, ..."
23,25,Doubs,"POLYGON ((6.80701 47.56280, 6.81666 47.54792, ..."
85,85,Vendée,"MULTIPOLYGON (((-2.30479 46.70942, -2.33559 46..."


Notice how `depts` is a geopandas dataframe.  We'll use it just as a regular `pandas` dataframe, but it includes the geometry info we need to be able to draw those regions when we pass them into Altair.  We just need to make sure that when we work with our data, we keep them in a geopandas dataframe and not a plain dataframe if we want to draw the departments.

In the next cell, notice how we do a right-merge to bring in department data into names.  We do this as a merge on `depts` because we need a geopandas dataframe.  Remember, `depts` is a geopandas dataframe, while `names` is a regular dataframe.  If we did a left merge on `names`, we'd end up with a regular pandas dataframe. After this merge, both `names` and `depts` will be geopandas dataframes.

**Hint:** Be careful when you do your data joins here.  It's easy to accidentally merge the wrong way to accidentally create a _much bigger_ dataset.

In [6]:
# Keep a reference around to the plain pandas dataframe, without geometry data, just in case
just_names = names

names = depts.merge(names, how='right', left_on='code', right_on='dpt')

names.sample(5)

,code_x,nom_x,geometry_x,code_y,nom_y,geometry_y,sexe,preusuel,annais,dpt,nombre
1107466,59,Nord,"MULTIPOLYGON (((3.04040 50.15971, 3.06301 50.1...",59,Nord,"MULTIPOLYGON (((3.04040 50.15971, 3.06301 50.1...",1,MARYAN,2000,59,3
1597417,06,Alpes-Maritimes,"POLYGON ((6.88743 44.36105, 6.92257 44.35073, ...",06,Alpes-Maritimes,"POLYGON ((6.88743 44.36105, 6.92257 44.35073, ...",1,VALENTIN,1988,06,6
93052,21,Côte-d'Or,"MULTIPOLYGON (((4.18190 47.15051, 4.18711 47.1...",21,Côte-d'Or,"MULTIPOLYGON (((4.18190 47.15051, 4.18711 47.1...",1,AMAURY,1988,21,4
1055967,56,Morbihan,"MULTIPOLYGON (((-3.42179 47.62000, -3.44067 47...",56,Morbihan,"MULTIPOLYGON (((-3.42179 47.62000, -3.44067 47...",1,MALO,2011,56,47
1679560,45,Loiret,"POLYGON ((2.93631 48.16339, 2.97126 48.15183, ...",45,Loiret,"POLYGON ((2.93631 48.16339, 2.97126 48.15183, ...",1,YOUNES,2017,45,11


# Show a name over all years

Now we'll choose a name to show across all years.  To that, we'll group all of the names in a department together (squashing the years together) and use the sum.

In [7]:
grouped = names.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum()
grouped = depts.merge(grouped, how='right', left_on='code', right_on='dpt') # Add geometry data back in
grouped

,code,nom,geometry,dpt,preusuel,sexe,nombre
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,AARON,1,160
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABBY,2,3
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDALLAH,1,7
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDEL,1,3
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDELKADER,1,3
...,...,...,...,...,...,...,...
239574,NaN,NaN,None,974,ÉSAÏE,1,3
239575,NaN,NaN,None,974,ÉTHAN,1,53
239576,NaN,NaN,None,974,ÉTIENNE,1,3
239577,NaN,NaN,None,974,ÉVA,2,32


Now let's pick a name and check out how it's distribution over the last 120 years across Metropolitan France.  In this example, I choose the name “Lucien,” which I rather like for some reason.

In [82]:
name = 'JOHN'
subset = grouped[grouped.preusuel == name]
alt.Chart(subset).mark_geoshape(stroke='white').encode(
    tooltip=['nom', 'code', 'nombre'],
    color='nombre',
).properties(width=800, height=600)

alt.Chart(...)

In [9]:
names.head()

,code_x,nom_x,geometry_x,code_y,nom_y,geometry_y,sexe,preusuel,annais,dpt,nombre
0,84,Vaucluse,"MULTIPOLYGON (((4.89291 44.36482, 4.90663 44.3...",84,Vaucluse,"MULTIPOLYGON (((4.89291 44.36482, 4.90663 44.3...",1,AADIL,1983,84,3
1,92,Hauts-de-Seine,"POLYGON ((2.29097 48.95097, 2.32697 48.94536, ...",92,Hauts-de-Seine,"POLYGON ((2.29097 48.95097, 2.32697 48.94536, ...",1,AADIL,1992,92,3
2,95,Val-d'Oise,"POLYGON ((2.59052 49.07965, 2.57203 49.06149, ...",95,Val-d'Oise,"POLYGON ((2.59052 49.07965, 2.57203 49.06149, ...",1,AAHIL,2016,95,3
3,75,Paris,"POLYGON ((2.41634 48.84924, 2.46226 48.84254, ...",75,Paris,"POLYGON ((2.41634 48.84924, 2.46226 48.84254, ...",1,AARON,1962,75,3
4,75,Paris,"POLYGON ((2.41634 48.84924, 2.46226 48.84254, ...",75,Paris,"POLYGON ((2.41634 48.84924, 2.46226 48.84254, ...",1,AARON,1976,75,3


In [10]:
names.describe()

,sexe,nombre
count,3.668274e+06,3.668274e+06
mean,1.536004e+00,2.079637e+01
std,4.987021e-01,5.648752e+01
min,1.000000e+00,3.000000e+00
25%,1.000000e+00,4.000000e+00
50%,2.000000e+00,7.000000e+00
75%,2.000000e+00,1.800000e+01
max,2.000000e+00,6.310000e+03


# Mini-Project

First, we have to create a DataFrame of names given to both sexes

In [54]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

In [55]:
GenderInt=pd.DataFrame(names, columns=['preusuel','sexe'])

In [56]:
GenderInt

,preusuel,sexe
10885,AADIL,1
10886,AADIL,1
10888,AAHIL,1
10892,AARON,1
10893,AARON,1
...,...,...
3727545,ZYA,2
3727546,ZYA,2
3727547,ZYA,2
3727548,ZYA,2


In [57]:
filtered_df1 = GenderInt.query('sexe == 1')
filtered_df1.drop_duplicates(keep = 'first', inplace=True)
filtered_df1

<ipython-input-57-c988d54ac1dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1.drop_duplicates(keep = 'first', inplace=True)


,preusuel,sexe
10885,AADIL,1
10888,AAHIL,1
10892,AARON,1
12391,AARONN,1
12394,AARUSH,1
...,...,...
1729898,ZYGFRYD,1
1729902,ZYGMOND,1
1729915,ZYGMONT,1
1729927,ZYGMUND,1


In [58]:
filtered_df2 = GenderInt.query('sexe == 2')
filtered_df2.drop_duplicates(keep = 'first', inplace=True)
filtered_df2

<ipython-input-58-5b8c64eab5c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df2.drop_duplicates(keep = 'first', inplace=True)


,preusuel,sexe
1741138,AALIYA,2
1741140,AALIYAH,2
1741462,AALYA,2
1741465,AALYAH,2
1741476,ABBIE,2
...,...,...
3727526,ZUMRA,2
3727531,ZÜMRA,2
3727535,ZUZANNA,2
3727542,ZYA,2


In [76]:
df3=pd.merge(filtered_df1['preusuel'],filtered_df2['preusuel'], how='inner')
df3

,preusuel
0,ABDON
1,ABEL
2,ACHILLE
3,ADAM
4,ADAMA
...,...
863,YVES
864,YVONNICK
865,ZACHARIE
866,ZAYANE


In [77]:
values=df3['preusuel']
filtered_df = names[names.preusuel.isin(values)]
filtered_df

,sexe,preusuel,annais,dpt,nombre
17141,1,ABDON,1901,59,4
17142,1,ABDON,1901,66,6
17143,1,ABDON,1903,66,5
17144,1,ABDON,1904,59,3
17145,1,ABDON,1904,66,3
...,...,...,...,...,...
3726186,2,ZOÉ,2020,95,27
3726187,2,ZOÉ,2020,971,8
3726188,2,ZOÉ,2020,972,5
3726189,2,ZOÉ,2020,973,7


This is the DataFrame we were looking for. Now, we are going to use Altair on this DataFrame

In [62]:
source = alt.pd.DataFrame(filtered_df)

In [72]:
source1=source.query('sexe==1')
source1

,sexe,preusuel,annais,dpt,nombre
17141,1,ABDON,1901,59,4
17142,1,ABDON,1901,66,6
17143,1,ABDON,1903,66,5
17144,1,ABDON,1904,59,3
17145,1,ABDON,1904,66,3
...,...,...,...,...,...
1729635,1,ZOÉ,1949,971,3
1729636,1,ZOÉ,1950,972,3
1729637,1,ZOÉ,1953,972,3
1729638,1,ZOÉ,1955,972,4


In [79]:
source2=source.query('sexe==2')
source2

,sexe,preusuel,annais,dpt,nombre
1741805,2,ABDON,1918,972,3
1741806,2,ABDON,1922,971,4
1741807,2,ABDON,1927,971,4
1741808,2,ABDON,1927,972,5
1741809,2,ABDON,1935,972,3
...,...,...,...,...,...
3726186,2,ZOÉ,2020,95,27
3726187,2,ZOÉ,2020,971,8
3726188,2,ZOÉ,2020,972,5
3726189,2,ZOÉ,2020,973,7


In [61]:
brush = alt.selection_interval(encodings=['x'])
click = alt.selection_multi(encodings=['color'])

In [101]:
line = alt.Chart(source).mark_bar(opacity=0.7).encode(
    alt.X('annais:T', axis=alt.Axis(format='%Y'),bin=alt.Bin(maxbins=250)),
    y=alt.Y('nombre',stack=None),
    color="sexe:N"
).properties(
    title="evolution of names given to both genders",
    width=1200,
    height=600
)

line

alt.Chart(...)

On this graph, we can see that fewer and fewer people have an existing first name in both genders. We can also see that there is no an equal distribution of "both genders" names : every time, there is a majority among male or female.